In [ ]:
from sys import path
path.append('../src')
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import newton
from sklearn.neighbors import KernelDensity
from scipy.io import savemat
from Davidenko import Davidenko

In [ ]:
% matplotlib inline

In [ ]:
minpos, maxpos, points = 0.9, 1.05, 1000
iters = 5
f = lambda x: (1 - 2 * x ** 6) / x ** 12
df = lambda x: 12 * (x ** 6 -1) / x ** 13
ddf = lambda x: (156 - 84 * x ** 6) / x ** 14
#f = lambda x: x ** 2 - x ** 4
#df = lambda x: 2 * x - 4 * x ** 3
#ddf = lambda x: 2 - 12 * x ** 2
newtonf = lambda x: newton(df, x, ddf, tol=100, maxiter=1)

In [ ]:
plt.plot(np.linspace(minpos, maxpos, points), f(np.linspace(minpos, maxpos, points)));
plt.ylim((-1.1, 2));

In [ ]:
newtondata = np.zeros((iters + 1, points))
newtondata[0, :] = np.linspace(minpos, maxpos, points)
for it in range(iters):
    for i, v in enumerate(newtondata[it, :]):
        newtondata[it + 1, i] = newtonf(v)

In [ ]:
trange = np.linspace(0, 3, 50)
davidenkodata = Davidenko(np.linspace(minpos, maxpos, points), df, ddf, trange=trange)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10,4))
for i in range(iters + 1):
    ax[0, 0].scatter(i * np.ones((newtondata.shape[1])), newtondata[i, :])
ax[1, 0].plot(newtondata)
for i, v in enumerate(trange):
    ax[0, 1].scatter(v * np.ones((davidenkodata.shape[0])), davidenkodata[:, i])
ax[1, 1].plot(trange, davidenkodata.T)
ax[0, 0].set_ylabel('Position'); ax[1, 0].set_ylabel('Positions');
ax[1, 0].set_xlabel('Iteration'); ax[1, 1].set_xlabel('Time');
fig.tight_layout()

In [ ]:
density = np.empty((newtondata.shape[0], newtondata.shape[1]))
samples = np.linspace(minpos, maxpos, points)
posdensity = np.empty((newtondata.shape[0], newtondata.shape[1]))
for i in range(newtondata.shape[0]):
    kde = KernelDensity(bandwidth=(20 * (max(newtondata[i, :]) - min(newtondata[i, :])) / points), kernel='gaussian')
    kde.fit(newtondata[i, :][:, np.newaxis])
    density[i, :] = np.exp(kde.score_samples(samples[:, np.newaxis]))
    #posdensity[:, i] = np.exp(kde.score_samples(positions[:, i][:, np.newaxis]))

In [ ]:
savemat('NewtonData', {'Positions' : newtondata, 'Density' : density})
savemat('DavidenkoData', {'Positions' : davidenkodata, 'Tsteps' : trange})